In [63]:
import json
import os
import sys
import pandas as pd
import numpy as np

sys.path.append("..")

files = ['leagues',
 'mapping_data',
 'players',
 'teams',
 'tournaments']


In [40]:
### Helper Functions

def create_handle_map(player_data)->dict:
    player_id_map = {entry['player_id']: entry['handle'] for entry in player_data}
    return player_id_map

def id_to_handle(indices, player_id_map):
    return {index: player_id_map[player_id] for index, player_id in indices.items() if player_id in player_id_map}


Esports Data:

1. leagues - ['name','slug','region','tournaments'] - name and region of the league and then within tournaments is a list of key-value pairs corresponding to {'id':"someid"} `tournaments` has the list of dicts corresponding to tournaments
2. mapping_data - ['esportsGameId','teamMapping','participantMapping'] - Game Id, dictionary containing {"200":"id","100":"id"},participant mapping has a dictionary containing {"3":"id"} for 1-10 as the keys `player_list` has the player ids for a game
3. players - ['player_id','handle','home_team_id'] - first and last name for a player is given too, maybe used to verify the player (esp if they changed handle). `player_map` has the player handles all in one reference
4. teams - ['team_id','name','acronym','slug'] - no nested information, can use this to directly give a name to each team
5. tournaments - 

In [74]:
file = files[4]

with open(f'C:/Users/akmar/PycharmProjects/lolpowerrank/riot_esports_ranking/esports-data/{file}.json', 'r') as f:
    data = json.load(f)

data[0]

{'id': '110733838935136200',
 'leagueId': '109518549825754242',
 'name': '#2 Summer 2023',
 'slug': 'nacl_qualifiers_2_summer_2023',
 'sport': 'lol',
 'startDate': '2023-07-20',
 'endDate': '2023-08-15',
 'stages': [{'name': 'Groups',
   'type': None,
   'slug': 'groups',
   'sections': [{'name': 'Group A',
     'matches': [{'id': '110733838936381392',
       'type': 'normal',
       'state': 'completed',
       'mode': 'classic',
       'strategy': {'type': 'bestOf', 'count': 3},
       'teams': [{'id': '109981650516317055',
         'side': 'blue',
         'record': {'wins': 2, 'losses': 0, 'ties': 0},
         'result': {'outcome': 'win', 'gameWins': 2},
         'players': [{'id': '101383793885656347', 'role': 'bottom'},
          {'id': '110536248987966005', 'role': 'jungle'},
          {'id': '109782987645004350', 'role': 'mid'},
          {'id': '108369185696299414', 'role': 'none'},
          {'id': '110536253876427328', 'role': 'none'},
          {'id': '98926509797315114', '

In [61]:
len(data)

674

In [48]:
player_list = []
for i in range(0,len(data),1000):
    player_list.append(data[i]["participantMapping"])

In [49]:
len(player_list)

26

In [41]:
player_map = create_handle_map(data)

In [73]:
player_map_df = pd.DataFrame.from_dict(player_map,'index')
player_map_df.columns = ["Player"]
player_map_df.to_csv("player_map.csv")

In [59]:
id_to_handle(player_list[2],player_map)

{'3': 'Bjergsen',
 '5': 'Biofrost',
 '10': 'IgNar',
 '2': 'Dardoch',
 '1': 'BrokenBlade',
 '9': 'WildTurtle',
 '7': 'Santorin',
 '8': 'PowerOfEvil',
 '6': 'Solo',
 '4': 'Kobbe'}

In [32]:
player_mappings = {}
for j in range(1,10):
    
for i in range(len(data)):
    if data[i]["player_id"] == 

4222

In [25]:
tournaments = []
for i in range(38):

    tournaments.append(data[i]['name'])

tournaments

['LCS',
 'LCS Challengers',
 'LCS Challengers Qualifiers',
 'College Championship',
 'CBLOL',
 'LCK',
 'LCL',
 'LCO',
 'LEC',
 'LJL',
 'LLA',
 'LPL',
 'PCS',
 'TCL',
 'VCS',
 'MSI',
 'Worlds',
 'All-Star Event',
 'EMEA Masters',
 'La Ligue Française',
 'NLC',
 'Elite Series',
 'Liga Portuguesa',
 'PG Nationals',
 'Ultraliga',
 'SuperLiga',
 'Prime League',
 'Hitpoint Masters',
 'Esports Balkan League',
 'Greek Legends League',
 'Arabian League',
 'LCK Academy',
 'LJL Academy',
 'LCK Challengers',
 'CBLOL Academy',
 'North Regional League',
 'South Regional League',
 'TFT Rising Legends']

In [14]:
def inspect_structure(d, indent=0):
    if isinstance(d, dict):
        for key, value in d.items():
            print('  ' * indent + f'Key: {key}, Type: {type(value)}')
            inspect_structure(value, indent + 1)
    elif isinstance(d, list) and len(d) > 0:
        print('  ' * indent + f'List of {type(d[0])} (length {len(d)})')
        inspect_structure(d[0], indent + 1)
        
inspect_structure(data)

List of <class 'dict'> (length 38)
  Key: id, Type: <class 'str'>
  Key: name, Type: <class 'str'>
  Key: slug, Type: <class 'str'>
  Key: sport, Type: <class 'str'>
  Key: image, Type: <class 'str'>
  Key: lightImage, Type: <class 'str'>
  Key: darkImage, Type: <class 'str'>
  Key: region, Type: <class 'str'>
  Key: priority, Type: <class 'int'>
  Key: displayPriority, Type: <class 'dict'>
    Key: position, Type: <class 'int'>
    Key: status, Type: <class 'str'>
  Key: tournaments, Type: <class 'list'>
    List of <class 'dict'> (length 25)
      Key: id, Type: <class 'str'>
